In [9]:
%pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

Note: you may need to restart the kernel to use updated packages.


Question 1. dlt Version

In [10]:
%pip show dlt

Name: dlt
Version: 1.13.0
Summary: dlt is an open-source python-first scalable data loading library that does not require any backend to run.
Home-page: https://github.com/dlt-hub
Author: 
Author-email: "dltHub Inc." <services@dlthub.com>
License-Expression: Apache-2.0
Location: c:\projects\DataTalksClub\llm-zoomcamp\.venv\Lib\site-packages
Requires: click, fsspec, gitpython, giturlparse, hexbytes, humanize, jsonpath-ng, orjson, packaging, pathvalidate, pendulum, pluggy, pytz, pywin32, pyyaml, requests, requirements-parser, rich-argparse, semver, setuptools, simplejson, sqlglot, tenacity, tomlkit, typing-extensions, tzdata, win-precise-time
Required-by: 
Note: you may need to restart the kernel to use updated packages.


dlt Resourse

In [12]:
import dlt
import requests

@dlt.resource(table_name="zoomcamp_docs", max_table_nesting=0)
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

Question 2. dlt pipeline

In [13]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [14]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Run started at 2025-07-11 14:28:50.581441+00:00 and COMPLETED in 37.94 seconds with 4 steps.
Step extract COMPLETED in 1.71 seconds.

Load package 1752244161.1925611 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.10 seconds.
Normalized data for the following tables:
- zoomcamp_docs: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1752244161.1925611 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 5.53 seconds.
Pipeline zoomcamp_pipeline load step completed in 5.50 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used c:\projects\DataTalksClub\llm-zoomcamp\cohorts\2025\workshops\dlt\db.qdrant location to store data
Load package 1752244161.1925611 is LOADED and contains no failed jobs

Step run COMPLETED in 37.94 seconds.
Pipeline zoomcamp_pipeline load step completed in 5.50 seconds
1 load p

Question 3. Embeddings

In [25]:
import json

with open("db.qdrant/meta.json", "r") as f:
    meta = json.load(f)

# Get the collections dictionary
collections = meta.get("collections", {})

# Loop through collections and extract the vector model names
embedding_models = set()
for collection in collections.values():
    vectors = collection.get("vectors", {})
    embedding_models.update(vectors.keys())

print("Embedding models used:", embedding_models)

Embedding models used: {'fast-bge-small-en'}
